In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import scipy.io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")


Using device: cpu


## Load the dataset and pre process
Note: here we use simulated linear positions for GNN (positions are used only for distance-based connectivity), while real 3D molecular coordinates are available for GSCNN (which requires accurate geometry)


In [2]:
data = scipy.io.loadmat('../qm7b.mat')

# Extract available data
X = data['X']  # Coulomb matrices: (7211, 23, 23)
T = data['T']  # Properties: (7211, 14)
names = data['names']  # Property names: (14,)

X = np.array(X)
T = np.array(T)

# Get property names
if names.ndim > 1:
    property_names = [str(names[i][0]) for i in range(len(names))]
else:
    property_names = [str(names[i]) for i in range(len(names))]

print(f"Dataset shapes:")
print(f"  X (Coulomb matrices): {X.shape}")
print(f"  T (Properties): {T.shape}")

# Extract atomic numbers from Coulomb matrix diagonals
# C_ii = 0.5 * Z_i^2.4, so Z_i = (2 * C_ii)^(1/2.4)
Z_list = []
for i in range(len(X)):
    coulomb_diag = np.diag(X[i])
    # Find non-zero diagonal elements (actual atoms)
    atom_indices = np.where(coulomb_diag > 1e-6)[0]
    if len(atom_indices) > 0:
        z_vals = (2 * coulomb_diag[atom_indices]) ** (1.0 / 2.4)
        z_vals = np.round(z_vals).astype(int)
        Z_list.append(z_vals)
    else:
        Z_list.append(np.array([1]))


R_list = []
for i, z_vals in enumerate(Z_list):
    n_atoms = len(z_vals)
    r_vals = np.zeros((n_atoms, 3))
    for j in range(n_atoms):
        r_vals[j, 0] = j * 1.5  # Simple spacing along x-axis
    R_list.append(r_vals)

Z = np.array(Z_list, dtype=object)
R = np.array(R_list, dtype=object)

print(f"Extracted Z for {len(Z)} molecules")


Dataset shapes:
  X (Coulomb matrices): (7211, 23, 23)
  T (Properties): (7211, 14)
Extracted Z for 7211 molecules


## GAT Architecture

Define GATConv (Graph Attention Network Convolution) and GAT model (embedding → GAT layers → global pooling → property prediction).


In [3]:
class GATConv(nn.Module):
    """Graph Attention Network Convolution Layer"""
    def __init__(self, n_in, n_out, n_heads=1, dropout=0.1):
        super(GATConv, self).__init__()
        self.n_in = n_in
        self.n_out = n_out
        self.n_heads = n_heads
        self.head_dim = n_out // n_heads
        
        # Linear transformation
        self.W = nn.Linear(n_in, n_out)
        
        # Attention mechanism: compute attention scores
        self.attn = nn.Linear(2 * self.head_dim, 1)
        
        self.dropout = nn.Dropout(dropout)
        self.leaky_relu = nn.LeakyReLU(0.2)
        
    def forward(self, x, edge_index, mask):
        """
        GAT convolution: attention-weighted aggregation
        """
        row, col = edge_index
        batch_size_n_atoms, n_in = x.shape
        
        # Linear transformation
        h = self.W(x)  # (batch_size_n_atoms, n_out)
        
        # Reshape for multi-head attention
        if self.n_heads > 1:
            h = h.view(batch_size_n_atoms, self.n_heads, self.head_dim)
        else:
            h = h.unsqueeze(1)  # (batch_size_n_atoms, 1, head_dim)
        
        # Compute attention scores for edges
        if len(row) > 0:
            # Get source and target node features
            h_src = h[row]  # (n_edges, n_heads, head_dim)
            h_tgt = h[col]  # (n_edges, n_heads, head_dim)
            
            # Concatenate source and target features
            h_cat = torch.cat([h_src, h_tgt], dim=-1)  # (n_edges, n_heads, 2*head_dim)
            
            # Compute attention scores
            attn_scores = self.attn(h_cat).squeeze(-1)  # (n_edges, n_heads)
            attn_scores = self.leaky_relu(attn_scores)
            
            # Apply mask to attention scores (mask out invalid edges)
            edge_mask = (mask[row] * mask[col]).unsqueeze(-1)  # (n_edges, 1)
            attn_scores = attn_scores * edge_mask + (1 - edge_mask) * (-1e9)
            
            # Softmax over neighbors for each node
            # Process each node separately for stable softmax
            out = torch.zeros(batch_size_n_atoms, self.n_heads, self.head_dim, device=x.device)
            
            # Group edges by source node
            for node_idx in range(batch_size_n_atoms):
                # Find all edges where this node is the source
                node_mask = (row == node_idx)
                if not node_mask.any():
                    continue
                
                # Get attention scores for this node's neighbors
                node_attn = attn_scores[node_mask]  # (n_neighbors, n_heads)
                
                # Apply softmax
                node_attn_weights = F.softmax(node_attn, dim=0)  # (n_neighbors, n_heads)
                node_attn_weights = node_attn_weights.unsqueeze(-1)  # (n_neighbors, n_heads, 1)
                
                # Get neighbor features
                neighbor_indices = col[node_mask]
                neighbor_features = h[neighbor_indices]  # (n_neighbors, n_heads, head_dim)
                
                # Weighted aggregation
                weighted = neighbor_features * node_attn_weights  # (n_neighbors, n_heads, head_dim)
                out[node_idx] = weighted.sum(dim=0)  # (n_heads, head_dim)
        else:
            out = h
        
        # Reshape back
        if self.n_heads > 1:
            out = out.view(batch_size_n_atoms, -1)  # (batch_size_n_atoms, n_out)
        else:
            out = out.squeeze(1)  # (batch_size_n_atoms, n_out)
        
        # Apply activation and dropout
        out = F.elu(out)
        out = self.dropout(out)
        
        # Apply mask
        out = out * mask.unsqueeze(-1)
        
        return out


class GAT(nn.Module):
    """Simplified GAT model for molecular property prediction"""
    def __init__(self, n_atom_basis=128, n_layers=2, n_heads=1,
                 n_out=14, max_atoms=23, dropout=0.1):
        super(GAT, self).__init__()
        self.n_atom_basis = n_atom_basis
        self.n_layers = n_layers
        self.max_atoms = max_atoms
        
        # Embedding layer for atomic numbers
        self.embedding = nn.Embedding(100, n_atom_basis)
        
        # GAT layers
        self.gat_layers = nn.ModuleList()
        for i in range(n_layers):
            self.gat_layers.append(
                GATConv(n_atom_basis, n_atom_basis, n_heads=n_heads, dropout=dropout)
            )
        
        # Simplified property prediction head
        self.property_head = nn.Sequential(
            nn.Linear(n_atom_basis, n_atom_basis // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(n_atom_basis // 2, n_out)
        )
        
    def build_edge_index(self, positions, mask, cutoff=5.0):
        batch_size, n_atoms, _ = positions.shape
        device = positions.device
        
        # Compute pairwise distances
        pos_i = positions.unsqueeze(2)
        pos_j = positions.unsqueeze(1)
        diff = pos_i - pos_j
        distances = torch.norm(diff, dim=-1)
        
        mask_i = mask.unsqueeze(2)
        mask_j = mask.unsqueeze(1)
        neighbor_mask = (mask_i * mask_j).float()
        
        eye = torch.eye(n_atoms, device=device).unsqueeze(0)
        neighbor_mask = neighbor_mask * (1 - eye)
        
        edge_mask = (distances < cutoff) * neighbor_mask
        
        # Build edge index for each batch
        edge_indices = []
        for b in range(batch_size):
            edges = torch.nonzero(edge_mask[b], as_tuple=False)
            if len(edges) > 0:
                edges_offset = edges + b * n_atoms
                edge_indices.append(edges_offset.t())
            else:
                self_loop = torch.arange(n_atoms, device=device) + b * n_atoms
                edge_indices.append(torch.stack([self_loop, self_loop]))
        
        if len(edge_indices) > 0:
            edge_index = torch.cat(edge_indices, dim=1)
        else:
            # Fallback: create self-loops
            edge_index = torch.stack([
                torch.arange(batch_size * n_atoms, device=device),
                torch.arange(batch_size * n_atoms, device=device)
            ])
        
        return edge_index
    
    def forward(self, atomic_numbers, positions, mask):
        batch_size, n_atoms, _ = positions.shape
        x = self.embedding(atomic_numbers)
        
        edge_index = self.build_edge_index(positions, mask)
        x_flat = x.view(-1, self.n_atom_basis)
        mask_flat = mask.view(-1)
        
        # Apply GAT layers
        for gat_layer in self.gat_layers:
            x_flat = gat_layer(x_flat, edge_index, mask_flat)
        
        # Global pooling: mean over atoms
        x = x_flat.view(batch_size, n_atoms, self.n_atom_basis)
        mask_expanded = mask.unsqueeze(-1).float()
        x_masked = x * mask_expanded
        x_sum = x_masked.sum(dim=1)
        n_atoms = mask.sum(dim=1, keepdim=True).float()
        x_mean = x_sum / (n_atoms + 1e-8)
        
        # Direct property prediction
        properties = self.property_head(x_mean)
        
        return properties


In [4]:
class QM7bDataset(Dataset):
    def __init__(self, Z, R, T, max_atoms=23):
        self.Z = Z
        self.R = R
        self.T = T
        self.n_molecules = len(Z)
        self.max_atoms = max_atoms
        self.n_properties = T.shape[1] if T.ndim > 1 else 1
        
    def __len__(self):
        return self.n_molecules
    
    def __getitem__(self, idx):
        z = torch.from_numpy(self.Z[idx]).long()
        r = torch.from_numpy(self.R[idx]).float()
        t = torch.from_numpy(self.T[idx]).float() if self.T.ndim > 1 else torch.tensor([self.T[idx]]).float()
        
        mask = (z > 0).float()
        
        return z, r, mask, t


## Data Preprocessing 
Pad/truncate molecules to fixed size (23 atoms max) for batch processing. Handle variable-length molecular data.


In [5]:
def prepare_molecular_data(Z, R, max_atoms=23):
    n_molecules = len(Z)
    
    Z_processed = []
    R_processed = []
    
    for i in range(n_molecules):
        z_vals = Z[i] if isinstance(Z[i], np.ndarray) else np.array([Z[i]])
        z_vals = z_vals.flatten()
        
        r_vals = R[i] if isinstance(R[i], np.ndarray) else np.zeros((len(z_vals), 3))
        if r_vals.ndim == 1:
            if len(r_vals) >= len(z_vals) * 3:
                r_vals = r_vals[:len(z_vals)*3].reshape(-1, 3)
            else:
                r_vals = np.zeros((len(z_vals), 3))
        elif r_vals.ndim == 2 and r_vals.shape[1] != 3:
            r_vals = np.zeros((len(z_vals), 3))
        
        if r_vals.shape[0] < len(z_vals):
            r_padded_temp = np.zeros((len(z_vals), 3))
            r_padded_temp[:r_vals.shape[0]] = r_vals
            r_vals = r_padded_temp
        elif r_vals.shape[0] > len(z_vals):
            r_vals = r_vals[:len(z_vals)]
        
        n_atoms = min(len(z_vals), max_atoms)
        z_padded = np.zeros(max_atoms, dtype=np.int64)
        r_padded = np.zeros((max_atoms, 3), dtype=np.float32)
        
        z_padded[:n_atoms] = z_vals[:n_atoms]
        r_padded[:n_atoms] = r_vals[:n_atoms]
        
        Z_processed.append(z_padded)
        R_processed.append(r_padded)
    
    return np.array(Z_processed), np.array(R_processed)

Z_processed, R_processed = prepare_molecular_data(Z, R, max_atoms=23)


In [6]:
# Split data to 80% train, 10% validation, 10% test
train_idx, temp_idx = train_test_split(
    np.arange(len(T)), test_size=0.2, random_state=42
)
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, random_state=42
)

# Create datasets
train_dataset = QM7bDataset(
    Z_processed[train_idx], 
    R_processed[train_idx], 
    T[train_idx],
    max_atoms=23
)
val_dataset = QM7bDataset(
    Z_processed[val_idx], 
    R_processed[val_idx], 
    T[val_idx],
    max_atoms=23
)
test_dataset = QM7bDataset(
    Z_processed[test_idx], 
    R_processed[test_idx], 
    T[test_idx],
    max_atoms=23
)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Train samples: 5768
Validation samples: 721
Test samples: 722


In [7]:
model = GAT(
    n_atom_basis=128,
    n_layers=2,
    n_heads=1,  # Single head for simplicity
    n_out=T.shape[1],  # Predict all 14 properties
    max_atoms=23,
    dropout=0.1
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=10
)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


Model parameters: 55,504


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device, epoch=None, total_epochs=None):
    model.train()
    total_loss = 0.0
    total_mae = 0.0
    n_batches = 0
    
    for batch_idx, (z, r, mask, t) in enumerate(loader):
        z = z.to(device)
        r = r.to(device)
        mask = mask.to(device)
        t = t.to(device)
        
        optimizer.zero_grad()
        predictions = model(z, r, mask)
        loss = criterion(predictions, t)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # Calculate MAE for this batch
        mae = torch.mean(torch.abs(predictions - t)).item()
        
        total_loss += loss.item()
        total_mae += mae
        n_batches += 1
    
    if epoch is not None:
        print()
    
    return total_loss / n_batches, total_mae / n_batches


def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_mae = 0.0
    n_batches = 0
    
    with torch.no_grad():
        for z, r, mask, t in loader:
            z = z.to(device)
            r = r.to(device)
            mask = mask.to(device)
            t = t.to(device)
            
            predictions = model(z, r, mask)
            loss = criterion(predictions, t)
            
            # Calculate MAE for this batch
            mae = torch.mean(torch.abs(predictions - t)).item()
            
            total_loss += loss.item()
            total_mae += mae
            n_batches += 1
    
    return total_loss / n_batches, total_mae / n_batches


# Training loop
n_epochs = 100
train_losses = []
train_maes = []
val_losses = []
val_maes = []
best_val_loss = float('inf')
best_val_mae = float('inf')
patience_counter = 0
max_patience = 20
current_lr = optimizer.param_groups[0]['lr']

print("Starting Training")

for epoch in range(n_epochs):
    # Training phase
    print(f"Epoch [{epoch+1}/{n_epochs}]")
    print(f"  Training...", end=' ')
    train_loss, train_mae = train_epoch(model, train_loader, criterion, optimizer, device, epoch+1, n_epochs)
    train_losses.append(train_loss)
    train_maes.append(train_mae)
    
    # Validation phase
    print(f"  Validating...", end=' ')
    val_loss, val_mae = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_maes.append(val_mae)
    
    # Update learning rate
    old_lr = current_lr
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    lr_reduced = old_lr != current_lr
    
    # Early stopping check (using MSE loss)
    improved = val_loss < best_val_loss
    if improved:
        best_val_loss = val_loss
        best_val_mae = val_mae
        patience_counter = 0
        # Save best model
        torch.save(model.state_dict(), '../best_gat_model.pth')
        save_status = "Saved"
    else:
        patience_counter += 1
        save_status = ""
    
    # Print epoch results
    print(f"  Results:")
    print(f"    Train MSE:  {train_loss:.6f}  |  Train MAE:  {train_mae:.6f}")
    print(f"    Val MSE:    {val_loss:.6f}  |  Val MAE:    {val_mae:.6f}")
    print(f"    LR:          {current_lr:.6f}")
    print()
    
    # Early stopping
    if patience_counter >= max_patience:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

print("Training completed")
print(f"Total epochs trained: {len(train_losses)}")
print(f"Best validation MSE: {best_val_loss:.6f}")
print(f"Best validation MAE: {best_val_mae:.6f}")


Starting Training
Epoch [1/100]
  Training... 
  Validating...   Results:
    Train MSE:  61456.887494  |  Train MAE:  64.116985
    Val MSE:    4486.390402  |  Val MAE:    18.152570
    LR:          0.001000

Epoch [2/100]
  Training... 

In [ ]:
# Plot training curves
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

# MAE curves
ax.plot(train_maes, label='Train MAE', linewidth=2)
ax.plot(val_maes, label='Validation MAE', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('MAE')
ax.set_title('GAT Training Curves - MAE')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
print("Evaluating the model on the test set")

model.load_state_dict(torch.load('../best_gat_model.pth'))
model.eval()

# Evaluate on test set
test_loss, test_mae = validate(model, test_loader, criterion, device)

print(f"Test MSE:  {test_loss:.6f}")
print(f"Test MAE:  {test_mae:.6f}")
